# Portfolio Workflow

In [4]:
from platform import python_version
import time
from datetime import datetime, timedelta
import os
import pandas as pd
import pandas_datareader as pdr

import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, 'tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import trading_factors as alpha_factors
importlib.reload(alpha_factors)
import utils
importlib.reload(utils)
import nonoverlapping_estimator as ai_estimator
importlib.reload(ai_estimator)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')
print(f'Pandas Data Reader version: {pdr.__version__}')

Sci-Kit version: 0.24.1
Python version: 3.8.8
Pandas version: 1.2.4
Pandas Data Reader version: 0.10.0


In [2]:
# For Chromedriver
from pathlib import Path

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories_yahoo.csv'

# Stage 1: Generate Stock Universe

- Gather stocks from specific criteria (SP500 top 50...)
- Use stock sentiment to select stocks
- Gather price histories

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 5 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [3]:
snp_500_stocks = utils.get_snp500()
stock_universe = utils.reduce_universe_by_sentiment(set(snp_500_stocks.index.to_list()))

Number of stocks in universe: 505


Tickers:   0%|          | 0/505 [00:00<?, ?Finvis Postings/s]

News Tables:   0%|          | 0/502 [00:00<?, ?News Table Items/s]

Mean Sentiment: 3.015000698918607 with a standared deviation of: 1.6622652797303485 providing a cutoff of: 1.3527354191882586
New number of stocks in universe: 423


In [4]:
number_of_years = 5
price_histories = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid').get_price_histories(
    stock_universe, datetime.today().strftime('%Y-%m-%d'),
    num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

Tickers:   0%|          | 0/439 [00:00<?, ?Price Histories/s]

In [11]:
from dateutil.relativedelta import relativedelta
number_of_years = 5
start = datetime.today() - relativedelta(years = number_of_years)
yahoo = pdr.yahoo.daily.YahooDailyReader(symbols=stock_universe, start=start, adjust_price=True, interval='d', get_actions=False, adjust_dividends=True)
price_histories = yahoo.read()
yahoo.close()
price_histories.to_csv(price_histories_file_name, index=True)

In [13]:
price_histories

Attributes Adj_Ratio                                                         \
Symbols            A       AAP      ABBV       ABC ABMD       ABT       ACN   
Date                                                                          
2017-02-10  0.960623  0.971871  0.794469  0.916967  1.0  0.917953  0.922028   
2017-02-13  0.960623  0.971871  0.794468  0.916967  1.0  0.917953  0.922028   
2017-02-14  0.960623  0.971871  0.794469  0.916967  1.0  0.917953  0.922028   
2017-02-15  0.960623  0.971871  0.794469  0.916967  1.0  0.917953  0.922028   
2017-02-16  0.960623  0.971871  0.794469  0.920653  1.0  0.917953  0.922028   
...              ...       ...       ...       ...  ...       ...       ...   
2022-02-03  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-04  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-07  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-08  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-09  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   

Attributes                           ...     Volume                        \
Symbols    ADBE       ADI       ADM  ...        WTW         WY        XEL   
Date                                 ...                                    
2017-02-10  1.0  0.905551  0.854681  ...   909900.0  4083900.0  1896600.0   
2017-02-13  1.0  0.905551  0.854681  ...   714800.0  7372400.0  2351600.0   
2017-02-14  1.0  0.905551  0.854681  ...   650500.0  4172600.0  2110800.0   
2017-02-15  1.0  0.905551  0.854681  ...   490800.0  4667900.0  1918500.0   
2017-02-16  1.0  0.905551  0.860876  ...   524500.0  3402200.0  2008500.0   
...         ...       ...       ...  ...        ...        ...        ...   
2022-02-03  1.0  1.000000  0.994717  ...  1032400.0  3698900.0  2530600.0   
2022-02-04  1.0  1.000000  0.994717  ...   807100.0  3244100.0  2517300.0   
2022-02-07  1.0  1.000000  1.000000  ...  1098100.0  3062000.0  2652900.0   
2022-02-08  1.0  1.000000  1.000000  ...  2036500.0  3893200.0  4155200.0   
2022-02-09  1.0  1.000000  1.000000  ...  1632911.0  4007921.0  3187690.0   

Attributes                                                                   \
Symbols          XLNX        XYL        YUM        ZBH      ZBRA       ZION   
Date                                                                          
2017-02-10  5044400.0  1409900.0  3190600.0   780500.0  184700.0  2458800.0   
2017-02-13  3862300.0  1606100.0  4407200.0   659900.0  172500.0  2197200.0   
2017-02-14  2394200.0  1747700.0  1946000.0  1129700.0  200800.0  2456500.0   
2017-02-15  2528700.0  1303200.0  1590300.0  1160200.0  214000.0  2690800.0   
2017-02-16  2231700.0  1183000.0  1968900.0  1490200.0  369700.0  1977400.0   
...               ...        ...        ...        ...       ...        ...   
2022-02-03  4842100.0  3649700.0  1172600.0  1871200.0  487800.0  1243100.0   
2022-02-04  3193600.0  2564500.0  1381100.0  1754900.0  394600.0  1207400.0   
2022-02-07  3259900.0  1330000.0  1666600.0  6293600.0  299800.0   809400.0   
2022-02-08  4838300.0  1750800.0  2345200.0  4522000.0  615200.0  1998800.0   
2022-02-09  6882024.0  1838597.0  3684499.0  2095234.0  678649.0  1351689.0   

Attributes              
Symbols            ZTS  
Date                    
2017-02-10   6160900.0  
2017-02-13   5636000.0  
2017-02-14   5216300.0  
2017-02-15   5639900.0  
2017-02-16  12881800.0  
...                ...  
2022-02-03   1734500.0  
2022-02-04   1919800.0  
2022-02-07   3303900.0  
2022-02-08   2880500.0  
2022-02-09   2007821.0  

[1259 rows x 2538 columns]

In [21]:
test_read = pd.read_csv(price_histories_file_name, header=[0, 1], index_col=[0], low_memory=False)

In [22]:
test_read

Attributes Adj_Ratio                                                         \
Symbols            A       AAP      ABBV       ABC ABMD       ABT       ACN   
Date                                                                          
2017-02-10  0.960623  0.971871  0.794469  0.916967  1.0  0.917953  0.922028   
2017-02-13  0.960623  0.971871  0.794468  0.916967  1.0  0.917953  0.922028   
2017-02-14  0.960623  0.971871  0.794469  0.916967  1.0  0.917953  0.922028   
2017-02-15  0.960623  0.971871  0.794469  0.916967  1.0  0.917953  0.922028   
2017-02-16  0.960623  0.971871  0.794469  0.920653  1.0  0.917953  0.922028   
...              ...       ...       ...       ...  ...       ...       ...   
2022-02-03  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-04  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-07  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-08  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   
2022-02-09  1.000000  1.000000  1.000000  1.000000  1.0  1.000000  1.000000   

Attributes                           ...     Volume                        \
Symbols    ADBE       ADI       ADM  ...        WTW         WY        XEL   
Date                                 ...                                    
2017-02-10  1.0  0.905551  0.854681  ...   909900.0  4083900.0  1896600.0   
2017-02-13  1.0  0.905551  0.854681  ...   714800.0  7372400.0  2351600.0   
2017-02-14  1.0  0.905551  0.854681  ...   650500.0  4172600.0  2110800.0   
2017-02-15  1.0  0.905551  0.854681  ...   490800.0  4667900.0  1918500.0   
2017-02-16  1.0  0.905551  0.860876  ...   524500.0  3402200.0  2008500.0   
...         ...       ...       ...  ...        ...        ...        ...   
2022-02-03  1.0  1.000000  0.994717  ...  1032400.0  3698900.0  2530600.0   
2022-02-04  1.0  1.000000  0.994717  ...   807100.0  3244100.0  2517300.0   
2022-02-07  1.0  1.000000  1.000000  ...  1098100.0  3062000.0  2652900.0   
2022-02-08  1.0  1.000000  1.000000  ...  2036500.0  3893200.0  4155200.0   
2022-02-09  1.0  1.000000  1.000000  ...  1632911.0  4007921.0  3187690.0   

Attributes                                                                   \
Symbols          XLNX        XYL        YUM        ZBH      ZBRA       ZION   
Date                                                                          
2017-02-10  5044400.0  1409900.0  3190600.0   780500.0  184700.0  2458800.0   
2017-02-13  3862300.0  1606100.0  4407200.0   659900.0  172500.0  2197200.0   
2017-02-14  2394200.0  1747700.0  1946000.0  1129700.0  200800.0  2456500.0   
2017-02-15  2528700.0  1303200.0  1590300.0  1160200.0  214000.0  2690800.0   
2017-02-16  2231700.0  1183000.0  1968900.0  1490200.0  369700.0  1977400.0   
...               ...        ...        ...        ...       ...        ...   
2022-02-03  4842100.0  3649700.0  1172600.0  1871200.0  487800.0  1243100.0   
2022-02-04  3193600.0  2564500.0  1381100.0  1754900.0  394600.0  1207400.0   
2022-02-07  3259900.0  1330000.0  1666600.0  6293600.0  299800.0   809400.0   
2022-02-08  4838300.0  1750800.0  2345200.0  4522000.0  615200.0  1998800.0   
2022-02-09  6882024.0  1838597.0  3684499.0  2095234.0  678649.0  1351689.0   

Attributes              
Symbols            ZTS  
Date                    
2017-02-10   6160900.0  
2017-02-13   5636000.0  
2017-02-14   5216300.0  
2017-02-15   5639900.0  
2017-02-16  12881800.0  
...                ...  
2022-02-03   1734500.0  
2022-02-04   1919800.0  
2022-02-07   3303900.0  
2022-02-08   2880500.0  
2022-02-09   2007821.0  

[1259 rows x 2538 columns]